### 1) Import Necessary Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

### 2) Loading and Preparing Dataset

In [2]:
# import dataset
loan_data = pd.read_csv('preprocessed_loans50k.csv')

# Convert 'status' to numerical labels
status_mapping = {
    'Safe': 0,
    'Risky': 1
}

loan_data['status'] = loan_data['status'].map(status_mapping)

# Separate features and labels
X = loan_data.drop('status', axis=1)
y = loan_data['status']

# Use LabelEncoder to encode 'Safe' as 0 and 'Risky' as 1
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader for training and testing data
train_dataset = data.TensorDataset(X_train, y_train)
test_dataset = data.TensorDataset(X_test, y_test)

train_loader = data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=64, shuffle=False)


### 3) Defining Binary Classification Model

In [3]:
class LoanBinaryModel(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(LoanBinaryModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out

### 4) Training Binary Classification Model

In [5]:

from torch.utils.tensorboard import SummaryWriter


def train_binary_model(model, train_loader, test_loader, num_epochs, learning_rate, target_accuracy, device):
    model.to(device)
    criterion = nn.BCELoss()
    optimizer = optim.Adadelta(model.parameters(), lr=learning_rate)
    writer = SummaryWriter()

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        average_loss = total_loss / len(train_loader)
        writer.add_scalar('Loss/train', average_loss, epoch + 1)

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                outputs = model(inputs)
                predicted = (outputs >= 0.5).float()
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        writer.add_scalar('Accuracy/test', accuracy, epoch + 1)

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%')

        if accuracy >= target_accuracy:
            print(f"Reached target accuracy of {target_accuracy:.2f}%.")
            break

    writer.flush()
    writer.close()

# Set the device to GPU if available, otherwise CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the model
input_dim = X_train.shape[1]
hidden_dim = 16
model = LoanBinaryModel(input_dim, hidden_dim)

# Train the model with stopping criterion
num_epochs = 100
learning_rate = 0.1
target_accuracy = 93.0  # Stop training when the accuracy reaches 95%
train_binary_model(model, train_loader, test_loader, num_epochs, learning_rate, target_accuracy, device)

Epoch [1/100], Loss: 32.3539, Accuracy: 83.05%
Epoch [2/100], Loss: 16.2451, Accuracy: 83.19%
Epoch [3/100], Loss: 16.1865, Accuracy: 83.14%
Epoch [4/100], Loss: 16.0272, Accuracy: 83.24%
Epoch [5/100], Loss: 15.9513, Accuracy: 83.17%
Epoch [6/100], Loss: 15.8081, Accuracy: 83.19%
Epoch [7/100], Loss: 15.5960, Accuracy: 78.04%
Epoch [8/100], Loss: 15.0503, Accuracy: 83.73%
Epoch [9/100], Loss: 14.4941, Accuracy: 83.43%
Epoch [10/100], Loss: 12.4496, Accuracy: 85.60%
Epoch [11/100], Loss: 11.0439, Accuracy: 86.73%
Epoch [12/100], Loss: 10.3214, Accuracy: 80.22%
Epoch [13/100], Loss: 10.4136, Accuracy: 78.08%
Epoch [14/100], Loss: 10.1462, Accuracy: 87.41%
Epoch [15/100], Loss: 9.8995, Accuracy: 87.86%
Epoch [16/100], Loss: 9.7344, Accuracy: 88.42%
Epoch [17/100], Loss: 9.0420, Accuracy: 89.23%
Epoch [18/100], Loss: 7.8741, Accuracy: 76.58%
Epoch [19/100], Loss: 7.2867, Accuracy: 83.87%
Epoch [20/100], Loss: 7.2344, Accuracy: 90.45%
Epoch [21/100], Loss: 6.8892, Accuracy: 90.06%
Epoch [2

### 5) Confusion Matrix

In [6]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

def evaluate_model(model, test_loader, device):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
            outputs = model(inputs)
            predicted = (outputs >= 0.5).float()  # Convert probabilities to binary predictions
            y_pred.extend(predicted.cpu().numpy())
            y_true.extend(labels.cpu().numpy())

    # Convert lists to numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Create the confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    print("Confusion Matrix:")
    print(cm)

    # Print classification report with zero_division parameter
    report = classification_report(y_true, y_pred, target_names=['Safe', 'Risky'], zero_division=1)
    print("\nClassification Report:")
    print(report)

# Evaluate the trained model and print the confusion matrix
evaluate_model(model, test_loader, device)

Confusion Matrix:
[[7752   14]
 [ 704  873]]

Classification Report:
              precision    recall  f1-score   support

        Safe       0.92      1.00      0.96      7766
       Risky       0.98      0.55      0.71      1577

    accuracy                           0.92      9343
   macro avg       0.95      0.78      0.83      9343
weighted avg       0.93      0.92      0.91      9343



### hyperparameter tuning

from torch.utils.tensorboard import SummaryWriter

def train_binary_model_with_tuning(model, train_loader, test_loader, num_epochs, learning_rate, hidden_dim, optimizer_name, device):
    model.to(device)
    criterion = nn.BCELoss()
    optimizer = get_optimizer(optimizer_name, model.parameters(), lr=learning_rate)
    writer = SummaryWriter(comment=f'-lr_{learning_rate}-hidden_dim_{hidden_dim}-optimizer_{optimizer_name}')

    total_steps = len(train_loader)
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        average_loss = total_loss / len(train_loader)
        writer.add_scalar('Loss/train', average_loss, epoch + 1)

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                outputs = model(inputs)
                predicted = (outputs >= 0.5).float()
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        writer.add_scalar('Accuracy/test', accuracy, epoch + 1)

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%')

    writer.flush()
    writer.close()

def get_optimizer(optimizer_name, parameters, lr):
    if optimizer_name == 'adadelta':
        return optim.SGD(parameters, lr=lr)
    elif optimizer_name == 'rmsprop':
        return optim.RMSprop(parameters, lr=lr)
    else:
        raise ValueError(f'Optimizer {optimizer_name} not supported.')

# Hyperparameter tuning

# Define hyperparameter search space
learning_rates = [0.1, 0.01, 0.0001]
hidden_dims = [16, 32, 64]
optimizers = ['adadelta', 'rmsprop']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_dim = X_train.shape[1]
num_epochs = 100

# Perform grid search over the hyperparameter space
for lr in learning_rates:
    for hidden_dim in hidden_dims:
        for optimizer_name in optimizers:
            model = LoanBinaryModel(input_dim, hidden_dim)
            train_binary_model_with_tuning(model, train_loader, test_loader, num_epochs, lr, hidden_dim, optimizer_name, device)

            
